###Ingesta del archivo "country.json"

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

####Paso1 - Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
countries_schema = "countryId INT, countryIsoCode String, countryName STRING"

In [0]:
countries_df = spark.read \
                .schema(countries_schema) \
                .json(f"{bronze_folder_path}/{v_file_date}/country.json")

In [0]:
countries_df.printSchema()

In [0]:
display(countries_df)

####Paso2 - Eliminar columnas del DataFrame

In [0]:
countries_dropped_df = countries_df.drop("countryIsoCode")
display(countries_dropped_df)

####Paso3 - Cambiar nombre de las columnas y añadir "ingestion_date" y "environment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit


In [0]:
countries_final_df = add_ingestion_dat(countries_dropped_df) \
                        .withColumn("environment", lit(v_environment)) \
                        .withColumnsRenamed({"countryId": "country_id", "countryName": "country_name"}) \
                        .withColumn("file_date",lit(v_file_date))

In [0]:
display(countries_final_df)

####Paso4 - Escribir la salida en un formato "Parquet"

In [0]:
#countries_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/countries/")
countries_final_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.countries")

In [0]:
#countries_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/countries/")

In [0]:
%fs
ls /mnt/mymoviehistory/silver/countries/

In [0]:
#display(spark.read.parquet("/mnt/mymoviehistory/silver/countries/"))

In [0]:
dbutils.notebook.exit("Success")